# 📓 Notebook - Teori & Praktik Bab 2: Proyek Machine Learning End-to-End

Selamat datang di notebook panduan untuk Bab 2! Bab ini adalah bab yang paling penting untuk memahami alur kerja sebuah proyek Machine Learning dari awal hingga akhir. Di sini, kita akan mempraktikkan setiap langkah yang dijelaskan di buku menggunakan dataset **California Housing Prices**.

## Langkah 1 & 2: Memahami Masalah dan Mengambil Data

**Teori (Penjelasan LLM):**
Setiap proyek ML dimulai dengan tujuan bisnis. Untuk proyek ini, tujuannya adalah **membangun model yang dapat memprediksi harga median rumah di sebuah distrik di California** berdasarkan data sensus.

Ini adalah masalah:
- **Supervised Learning**: Karena data kita memiliki label (harga median rumah).
- **Regresi**: Karena kita memprediksi sebuah nilai numerik.
- **Batch Learning**: Karena kita akan melatih model menggunakan semua data yang ada sekaligus.

Kita akan mulai dengan mengambil data dan melihat strukturnya.

In [ ]:
# Setup awal: Impor library yang dibutuhkan
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tarfile
from six.moves import urllib

# Lokasi download data
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    """Fungsi untuk mengunduh dan mengekstrak data."""
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

def load_housing_data(housing_path=HOUSING_PATH):
    """Fungsi untuk memuat data dari file CSV."""
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

# Jalankan fungsi untuk mendapatkan data
fetch_housing_data()
housing = load_housing_data()

# Lihat 5 baris pertama dari data
print("Data 5 baris pertama:")
housing.head()

In [ ]:
# Lihat informasi umum tentang data
print("\nInformasi Dataset:")
housing.info()

## Langkah Penting: Membuat Test Set

**Teori (Penjelasan LLM):**
Sebelum kita melakukan eksplorasi data lebih jauh, kita **harus** memisahkan *test set*. Mengapa? Untuk menghindari **data snooping bias**.

Otak kita sangat pandai menemukan pola. Jika kita melihat *test set* terlalu awal, kita mungkin secara tidak sadar menemukan pola di sana dan memilih model yang bekerja baik untuk *test set* tersebut. Akibatnya, saat kita mengevaluasi model di akhir, hasilnya akan terlalu optimis dan tidak mencerminkan performa model di dunia nyata.

Kita akan menggunakan **Stratified Sampling** untuk memastikan *test set* kita representatif.

In [ ]:
from sklearn.model_selection import train_test_split

# Pertama, buat kategori pendapatan untuk stratified sampling
housing["income_cat"] = pd.cut(housing["median_income"], 
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf], 
                               labels=[1, 2, 3, 4, 5])

# Lakukan stratified split berdasarkan kategori pendapatan
strat_train_set, strat_test_set = train_test_split(
    housing, test_size=0.2, random_state=42, stratify=housing["income_cat"])

# Hapus kolom 'income_cat' agar data kembali seperti semula
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

print(f"Ukuran Training Set: {len(strat_train_set)}")
print(f"Ukuran Test Set: {len(strat_test_set)}")

# Mulai sekarang, kita hanya akan bekerja dengan 'strat_train_set'.
# Kita akan "melupakan" 'strat_test_set' sampai akhir.

## Langkah 3: Menjelajahi dan Memvisualisasikan Data

**Teori (Penjelasan LLM):**
Tujuan dari fase ini adalah untuk mendapatkan *insight* atau pemahaman mendalam tentang data. Dengan visualisasi, kita bisa menemukan korelasi, pola, dan anomali yang mungkin tidak terlihat dari sekadar melihat angka.

Kita akan membuat beberapa plot untuk memahami distribusi geografis dan hubungan antar fitur.

In [ ]:
# Buat salinan training set agar data asli tidak termodifikasi
housing = strat_train_set.copy()

# Visualisasi geografis
# Parameter 'alpha=0.1' membantu melihat area padat penduduk
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1, figsize=(10,7))
plt.title("Distribusi Geografis Data")
plt.show()

In [ ]:
# Visualisasi harga rumah (warna) dan populasi (ukuran)
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
             s=housing["population"]/100, label="population", figsize=(10,7),
             c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True)
plt.legend()
plt.title("Harga Rumah vs Lokasi & Populasi")
plt.show()

# Insight: Harga rumah sangat berkaitan dengan lokasi (misalnya, dekat laut) dan kepadatan penduduk.

### Mencari Korelasi

**Teori (Penjelasan LLM):**
Kita bisa menghitung koefisien korelasi standar (Pearson's r) antara setiap pasang atribut. Korelasi berkisar dari -1 (korelasi negatif kuat) hingga 1 (korelasi positif kuat). Nilai mendekati 0 berarti tidak ada korelasi linear.

In [ ]:
corr_matrix = housing.corr()

# Lihat korelasi setiap fitur dengan harga median rumah
print(corr_matrix["median_house_value"].sort_values(ascending=False))

### Eksperimen dengan Kombinasi Atribut

**Teori (Penjelasan LLM):**
Terkadang, kombinasi beberapa fitur bisa memberikan informasi yang lebih baik daripada fitur-fitur itu sendiri. Ini adalah bagian dari *feature engineering*. Misalnya, jumlah kamar per rumah mungkin lebih informatif daripada total kamar di satu distrik.

In [ ]:
housing["rooms_per_household"] = housing["total_rooms"] / housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"] / housing["total_rooms"]
housing["population_per_household"] = housing["population"] / housing["households"]

corr_matrix = housing.corr()
print(corr_matrix["median_house_value"].sort_values(ascending=False))

# Insight: Fitur baru 'rooms_per_household' dan 'bedrooms_per_room' ternyata memiliki korelasi yang lebih baik.

## Langkah 4: Mempersiapkan Data untuk Algoritma ML

**Teori (Penjelasan LLM):**
Kebanyakan algoritma ML tidak bisa bekerja dengan data yang hilang atau fitur non-numerik. Kita juga perlu memastikan semua fitur memiliki skala yang sama. Proses ini harus diotomatisasi menggunakan fungsi atau *pipeline* agar mudah diterapkan pada data baru.

Kita akan melakukan:
1. **Data Cleaning**: Mengisi nilai yang hilang.
2. **Handling Text/Categorical**: Mengubah fitur teks menjadi angka.
3. **Feature Scaling**: Menyamakan skala semua fitur numerik.
4. **Transformation Pipelines**: Menggabungkan semua langkah menjadi satu.

In [ ]:
# Pisahkan kembali fitur (predictors) dan label (target)
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

# Pisahkan fitur numerik dan kategorikal
housing_num = housing.drop("ocean_proximity", axis=1)
housing_cat = housing[["ocean_proximity"]]

In [ ]:
# 1. Data Cleaning: Mengisi nilai null dengan median
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")
imputer.fit(housing_num) # imputer belajar median dari data numerik
X = imputer.transform(housing_num)
housing_tr = pd.DataFrame(X, columns=housing_num.columns, index=housing_num.index)

# 2. Handling Categorical: Menggunakan OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
print("Fitur kategorikal setelah di-encode (sparse matrix):")
print(housing_cat_1hot.toarray()[:5])

### Transformation Pipelines

**Teori (Penjelasan LLM):**
Scikit-Learn menyediakan `Pipeline` untuk merangkai beberapa langkah transformasi secara berurutan. Ini sangat berguna untuk menjaga kode tetap rapi dan mengurangi risiko kesalahan. `ColumnTransformer` memungkinkan kita menerapkan pipeline yang berbeda untuk kolom yang berbeda (misalnya, numerik dan kategorikal).

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

# Pipeline untuk fitur numerik
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('std_scaler', StandardScaler()),
])

# ColumnTransformer untuk menggabungkan semua transformasi
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", OneHotEncoder(), cat_attribs),
])

# Jalankan pipeline pada data housing
housing_prepared = full_pipeline.fit_transform(housing)

print("Shape data setelah diproses:", housing_prepared.shape)
print("Data siap digunakan untuk melatih model!")

## Langkah 5: Memilih dan Melatih Model

**Teori (Penjelasan LLM):**
Sekarang data kita sudah bersih dan siap pakai. Saatnya melatih beberapa model. Sebaiknya kita mulai dari model yang sederhana, seperti Regresi Linier, lalu coba model yang lebih kompleks.

Kita akan mengukur performa menggunakan **Root Mean Square Error (RMSE)**. Semakin kecil RMSE, semakin baik modelnya.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

# Evaluasi pada training set
housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
print(f"RMSE Regresi Linier pada Training Set: {lin_rmse:.2f}")

# Insight: Error sekitar $68,628. Ini cukup besar, artinya model ini underfitting.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
print(f"RMSE Decision Tree pada Training Set: {tree_rmse:.2f}")

# Insight: Error 0.0! Ini adalah tanda jelas bahwa model ini overfitting.

### Evaluasi yang Lebih Baik Menggunakan Cross-Validation

**Teori (Penjelasan LLM):**
Karena model *Decision Tree* overfitting, kita butuh cara evaluasi yang lebih baik. **K-fold Cross-Validation** adalah solusinya. Ia membagi training set menjadi beberapa bagian (misalnya 10), lalu melatih dan mengevaluasi model 10 kali, setiap kali menggunakan bagian yang berbeda sebagai set validasi. Ini memberikan estimasi performa yang lebih stabil.

In [ ]:
from sklearn.model_selection import cross_val_score

# Scikit-learn cross-validation mengharapkan fungsi utility (semakin besar semakin baik),
# bukan fungsi loss (semakin kecil semakin baik), jadi kita gunakan 'neg_mean_squared_error'
scores = cross_val_score(tree_reg, housing_prepared, housing_labels, 
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(tree_rmse_scores)

# Insight: Rata-rata errornya sekitar $71,200. Ini jauh lebih buruk dari Regresi Linier. Decision Tree jelas overfitting.

## Langkah 6: Fine-Tune Model

**Teori (Penjelasan LLM):**
Setelah memilih beberapa model yang menjanjikan (misalnya, `RandomForestRegressor`), kita perlu mencari kombinasi *hyperparameter* terbaik. Mencoba satu per satu secara manual sangat melelahkan. **Grid Search** adalah teknik untuk mengotomatiskan proses ini. Ia akan mencoba semua kombinasi hyperparameter yang kita berikan dan menemukan yang terbaik melalui cross-validation.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# Tentukan kombinasi hyperparameter yang ingin dicoba
# Peringatan: Proses ini bisa memakan waktu cukup lama!
# Kita gunakan sedikit kombinasi agar cepat.
param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

grid_search.fit(housing_prepared, housing_labels)

print("Hyperparameter terbaik:", grid_search.best_params_)

# Lihat skor terbaik
best_rmse = np.sqrt(-grid_search.best_score_)
print(f"RMSE terbaik dari Grid Search: {best_rmse:.2f}")

## Langkah 7: Evaluasi Final pada Test Set

**Teori (Penjelasan LLM):**
Setelah menemukan model terbaik dan hyperparameter-nya, inilah saatnya kita melihat performa finalnya pada *test set* yang selama ini kita simpan. Hasil ini akan menjadi estimasi terbaik kita tentang bagaimana model akan bekerja pada data baru di dunia nyata.

In [ ]:
final_model = grid_search.best_estimator_

# Pisahkan fitur dan label dari test set
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

# Jalankan test set melalui pipeline (HANYA transform, JANGAN fit!)
X_test_prepared = full_pipeline.transform(X_test)

# Buat prediksi final
final_predictions = final_model.predict(X_test_prepared)

# Hitung RMSE final
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

print(f"RMSE Final pada Test Set: {final_rmse:.2f}")

## Kesimpulan

Selamat! Anda telah melalui seluruh alur kerja proyek Machine Learning:
1. Memahami masalah
2. Mengambil data
3. Membuat *test set* yang representatif
4. Mengeksplorasi data untuk mencari *insight*
5. Mempersiapkan data secara otomatis dengan *pipeline*
6. Melatih beberapa model dan memilih yang terbaik
7. Melakukan *fine-tuning* hyperparameter
8. Mengevaluasi model final pada *test set*

Langkah-langkah ini adalah fondasi yang akan Anda gunakan berulang kali dalam karier Machine Learning Anda.